<a href="https://colab.research.google.com/github/Kaaiokk/Projeto_faculdade_chatbot/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
from nltk.stem.porter import PorterStemmer
nltk.download('punkt_tab')

import json
import random
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [ ]:
stemmer = PorterStemmer()

def tokenize(sentence):
    """
    Separa a frase em um array de palavras/tokens.
    """
    return nltk.word_tokenize(sentence)

def stem(word):
    """
    Encontra a raiz da palavra (stemming).
    Ex: "organizing" -> "organ"
    """
    return stemmer.stem(word.lower())

def bag_of_words(tokenized_sentence, all_words):
    """
    Cria um vetor "bag of words" (saco de palavras).
    Retorna um array: 1 para cada palavra conhecida que existe na frase, 0 caso contrário.
    Exemplo:
    sentence = ["hello", "how", "are", "you"]
    all_words = ["hi", "hello", "I", "you", "bye", "thank", "cool"]
    bag       = [  0  ,    1  ,  0  ,   1  ,   0  ,     0   ,    0   ]
    """
    tokenized_sentence = [stem(w) for w in tokenized_sentence]

    bag = np.zeros(len(all_words), dtype=np.float32)
    for idx, w in enumerate(all_words):
        if w in tokenized_sentence:
            bag[idx] = 1.0

    return bag

In [ ]:
class NeuralNet(nn.Module):
    # GARANTA QUE ESTA LINHA ESTEJA CORRETA
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.l2 = nn.Linear(hidden_size, hidden_size)
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        # sem softmax no final, pois CrossEntropyLoss já aplica
        return out

In [ ]:
# 1. Carregar e pré-processar os dados
with open("intents.json", "r") as f:
    intents = json.load(f)

all_words = []
tags = []
xy = []

# Iterar sobre cada intenção e seus padrões
for intent in intents['intents']:
    tag = intent['tag']
    tags.append(tag)
    for pattern in intent['patterns']:
        w = tokenize(pattern)
        all_words.extend(w)
        xy.append((w, tag))

# Aplicar stemming e remover duplicatas
ignore_words = ['?', '!', '.', ',']
all_words = [stem(w) for w in all_words if w not in ignore_words]
all_words = sorted(set(all_words))
tags = sorted(set(tags))

# 2. Criar os dados de treinamento (X e Y)
x_train = []
y_train = []

for (pattern_sentence, tag) in xy:
    bag = bag_of_words(pattern_sentence, all_words)
    x_train.append(bag)

    label = tags.index(tag)
    y_train.append(label)

x_train = np.array(x_train, dtype=np.float32)
y_train = np.array(y_train, dtype=np.int64)

# 3. Criar o Dataset e DataLoader do PyTorch
class ChatDataset(Dataset):
    def __init__(self):
        self.num_samples = len(x_train)
        self.x_data = x_train
        self.y_data = y_train

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.num_samples

# Hyperparameters
batch_size = 8
hidden_size = 8
output_size = len(tags)
input_size = len(x_train[0])
learning_rate = 0.001
num_epochs = 1000

dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, num_workers=0) # num_workers=0 para evitar problemas no Windows

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = NeuralNet(input_size, hidden_size, output_size).to(device)

# Loss e otimizador
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# 4. Loop de treinamento
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(words)
        loss = criterion(outputs, labels)

        # Backward and optimizer step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Imprimir progresso
    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

print(f'Final loss: {loss.item():.4f}')

# 5. Salvar o modelo treinado
data = {
    "model_state": model.state_dict(),
    "input_size": input_size,
    "output_size": output_size,
    "hidden_size": hidden_size,
    "all_words": all_words,
    "tags": tags
}

FILE = "data.pth"
torch.save(data, FILE)

print(f'Treinamento completo. Arquivo salvo em {FILE}')

In [ ]:
#  Configuração do dispositivo (CPU ou GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

# Carrega as intenções do arquivo JSON
with open('intents.json', 'r') as f:
    intents = json.load(f)

# Carrega os dados do modelo treinado
FILE = "data.pth"
data = torch.load(FILE)

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data["all_words"]
tags = data["tags"]
model_state = data["model_state"]

# Recria o modelo com a mesma arquitetura e carrega os pesos salvos
model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()  # Coloca o modelo em modo de avaliação

# Inicia a conversa
bot_name = "Sam"
print("Vamos conversar! Digite 'sair' para terminar.")

while True:
    sentence = input("Você: ")
    if sentence.lower() == "sair":
        break

    # 1. Processa a frase do usuário
    sentence = tokenize(sentence)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])  # Adiciona uma dimensão de batch (lote)
    X = torch.from_numpy(X).to(device)

    # 2. Obtém a previsão do modelo
    output = model(X)
    _, predicted = torch.max(output, dim=1)
    tag = tags[predicted.item()]

    # 3. Verifica a confiança da previsão usando softmax
    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]

    # 4. Seleciona e exibe a resposta
    if prob.item() > 0.25:  # Limiar de confiança
        for intent in intents['intents']:
            if tag == intent["tag"]:
                print(f"{bot_name}: {random.choice(intent['responses'])}")
    else:
        print(f"{bot_name}: Não entendi... Pode reformular a sua pergunta?")